In [1]:
try:
    if is_init:
        pass
except:
    %pwd
    %cd ..
    %pwd
    is_init = True

/home/spencersong/metamaterials_ai/src


In [113]:
from pathlib import Path
from src.utils import FileUtils
from config import Config
from utils import rmse
import torch
import numpy as np
import random
import itertools
data_folder = '../local_data'
work_folder = '../local_work'
configs_folder = '../configs'

In [91]:
def denormalize_decode_result(y_hat, max_speed, max_spacing, min_speed, min_spacing):
    """input: 1,14 tensor
        output: 1,3 tensor with wattage no longer one hot encoded"""
    watts = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]

    watt_arg = torch.argmax(y_hat[0][2:])
    y_final = torch.empty(3, dtype=torch.float32)
    y_final[0] = max_speed * y_hat[0][0] # TODO call the scale
    y_final[1] = max_spacing * y_hat[0][1] # TODO call the scale
    y_final[2]= watts[watt_arg]
    return y_final

# Inverse

In [77]:
filepath = Path(
    f"{work_folder}/saved_best/best_I-resnet1d-stainless-2022-07-11_01-00.ckpt") #CHANGEME
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

../local_work/saved_best/best_I-resnet1d-stainless-2022-07-11_01-00.ckpt


In [85]:
def get_test_data(path):
    stain_data = torch.load(
                    Path(f"{data_folder}/{path}"))#["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
    wave_data = stain_data["interpolated_wavelength"]
    emiss_data = stain_data["interpolated_emissivity"]#.keys()#["wavelenght"]
    #print(wave_test.max(1), wave_test.min(1)) #2.5002, 11.9479
    #print(emiss_test.max(1)[0])#.max(1), emiss_test.min(1)) #0, 1
    wave_test = wave_data[round(len(wave_data) * .95):]  
    emiss_test = emiss_data[round(len(wave_data) * .95):]
    return wave_test, emiss_test
#print(max_wave, min_wave)

In [88]:
#pass test data into inverse model, get predicted parameters
from src.models import InverseModel
i_model = InverseModel.load_from_checkpoint(filepath, strict=False)
i_model.eval()

wave_test, emiss_test = get_test_data('stainless-steel-revised-shuffled.pt')
test_y_hats = []#torch.empty(size = len(emiss_test))
for emiss in emiss_test:
    with torch.no_grad():
        test_y_hats.append(i_model(emiss.reshape(1, 800)))

In [105]:
test_y_hats[0]

tensor([[0.1704, 0.3627, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])

# Direct Model

In [119]:
from src.models import DirectModel
direct_filepath = Path(
    f"{work_folder}/saved_best/best_D-ann-inconel-2022-07-05_14-39.ckpt") #CHANGEME
print(direct_filepath)
if not Path.is_file(direct_filepath):
    raise Exception(f'Model file does not exist at {direct_filepath}!')

../local_work/saved_best/best_D-ann-inconel-2022-07-05_14-39.ckpt


In [120]:
d_model = DirectModel.load_from_checkpoint(direct_filepath)
d_model.eval()
emiss_y_hats = []
for x in test_y_hats:
    with torch.no_grad():
        emiss_y_hats.append(d_model(x.reshape(1,14)))

In [121]:
#calculate rmse
pred_rmse_vals = []
for i in range(len(emiss_test)):
    pred_rmse_vals.append(rmse(emiss_y_hats[i], emiss_test[i]).item())
stdev = np.std(pred_rmse_vals)
print(f'Direct Filepath: {direct_filepath}')
print(f'Inverse Filepath: {filepath}')
print(f'SD of RMSEs on test data: {stdev}')


Direct Filepath: ../local_work/saved_best/best_D-ann-inconel-2022-07-05_14-39.ckpt
Inverse Filepath: ../local_work/saved_best/best_D-ann-inconel-2022-07-05_14-39.ckpt
SD of RMSEs on test data: 0.17008276292283978


/home/spencersong/metamaterials_ai/src/utils/utils.py:43: UserWarning: Using a target size (torch.Size([800])) that is different to the input size (torch.Size([1, 800])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return (torch.nn.functional.mse_loss(pred, target) + epsilon).sqrt()


In [ ]:
import altair

In [ ]:
#visualize predictions vs original

